# Tournament Simulation

- Robert Shaw
- Data Driven March Madness

In this file, we test code written in march_madness_classes. The code allows us to train a head to head model predicting the the probability of team 1 beating team 2 in a college basketball games based on a logistic model of the probability. We run the tournament n times, counting the expected score of each team over the iterations. We then take the total score over the n iterations and predict head to head matchups as arg_max(points_1, points_2).

In [1]:
import march_madness_classes as mmc
import march_madness_games as mmg
import march_madness_models as mmm
import pandas as pd

In [2]:
# read in the dataset
seeds = pd.read_csv("datasets/kaggle_data/TourneySeeds.csv")
slots = pd.read_csv("datasets/kaggle_data/TourneySlots.csv")
games = pd.read_csv("datasets/kaggle_data/TourneyCompactResults.csv")

seeds_arr = mmg.filter_into_seasons(seeds)
slots_arr = mmg.filter_into_seasons(slots)
games_arr = mmg.filter_into_seasons(games)

In [3]:
# extract predictors
markov_data = pd.read_csv("datasets/our_data/stationary", index_col=0)
consistency = pd.read_csv("datasets/our_data/consistency", index_col=0)
dominance = pd.read_csv("datasets/our_data/dominance", index_col=0)
past_resul = pd.read_csv("datasets/our_data/past_results", index_col=0)
rpi = pd.read_csv("datasets/our_data/rpi", index_col=0)
bad_losses = pd.read_csv("datasets/our_data/bad_losses", index_col=0)
tough_wins = pd.read_csv("datasets/our_data/tough_wins", index_col=0)
close_games= pd.read_csv("datasets/our_data/close_games", index_col=0)
close_wins = pd.read_csv("datasets/our_data/close_wins",index_col=0)
close_wins_perc = pd.read_csv("datasets/our_data/close_wins_perc", index_col=0)
momentum = pd.read_csv("datasets/our_data/momentum", index_col=0)
weighted_wins = pd.read_csv("datasets/our_data/weighted_wins", index_col=0)
seed_matrix_df = pd.read_csv("datasets/our_data/team_summary_data/seeds_matrix", index_col=0)

In [4]:
# get data into correct format
predictor_names = ["min_index_id", "max_index_id", "markov", "dominance", "rpi", "bad_losses", "tough_wins", "close_wins", "close_wins_perc", "weighted_wins", "past_resul", "momentum"] 
predictor_dfs = [markov_data, dominance, rpi, bad_losses, tough_wins, close_wins, close_wins_perc, weighted_wins, past_resul, momentum]                                           

In [5]:
pred, resp = mmg.generate_multiple_years_of_games(range(1987, 2001), 
                                                  seeds_arr, 
                                                  slots_arr, 
                                                  games_arr, 
                                                  ["min_index_id", "max_index_id", "markov", "dominance", "rpi", "bad_losses", "tough_wins", "close_wins", "close_wins_perc", "weighted_wins", "past_resul", "momentum"], 
                                                  [markov_data, dominance, rpi, bad_losses, tough_wins, close_wins, close_wins_perc, weighted_wins, past_resul, momentum],
                                                  scoring_dif = False)


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LogReg

In [7]:
scaler = StandardScaler().fit(pred.iloc[:, [2,4,5,6,7,10]])

log_reg_model = LogReg(C = 10)
log_reg_model.fit(scaler.transform(pred.iloc[:, [2,4,5,6,7,10]]), resp.values.T[0])

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Test on 2001

In [58]:
# get data needed
seeds_2001 = seeds_arr[2001-1985] 
slots_2001 = slots_arr[2001-1985] 
games_2001 = games_arr[2001-1985]
predictors = ["min_index_team", "max_index_team", "markov_data", "rpi", "bad_losses", "tough_wins", "close_wins", "past_resul"] 
predictor_dfs = [markov_data, rpi, bad_losses, tough_wins, close_wins, past_resul]

In [127]:
# setup head to head model based on the log_reg_model we just trained
head_to_head_model_2001 = mmm.ModelPredictor(log_reg_model, scaler, predictor_dfs, 2001, simulation=True)

# simulate 2001 setup
simulator_2001 = mmc.Simulator(seeds_2001, slots_2001, head_to_head_model_2001)

# simulate 2001 n times
points = simulator_2001.simulate_tournament(5)

# extract our bracket, predicted based on expected points scored
bracket = simulator_2001.predict_tournament()

In [128]:
actual_model= mmm.ActualTournament(games_arr[2001-1985])
actual_tourney = mmc.Tournament(seeds_2001, slots_2001, actual_model, include_scoring_dif=False)

In [129]:
simulator_2001.score_tournament(actual_tourney)

Total Points  : 1360

Total Accuracy: 42 / 63 = 0.666666666667
R1    Accuracy: 22 / 32 = 0.6875
R2    Accuracy: 9 / 16 = 0.5625
R3    Accuracy: 6 / 8 = 0.75
R4    Accuracy: 3 / 4 = 0.75
R5    Accuracy: 1 / 2 = 0.5
R6    Accuracy: 1 / 1 = 1.0


(1360, 0.6666666666666666)

In [96]:
simple_model = mmm.ModelPredictor(log_reg_model, scaler, predictor_dfs, 2001, simulation=False)
simple_tourney = mmc.Tournament(seeds_2001, slots_2001, simple_model, include_scoring_dif=False)

simple_tourney.score_tournament(actual_tourney, print_res=True)

Total Points  : 1280

Total Accuracy: 40 / 63 = 0.634920634921
R1    Accuracy: 20 / 32 = 0.625
R2    Accuracy: 10 / 16 = 0.625
R3    Accuracy: 6 / 8 = 0.75
R4    Accuracy: 2 / 4 = 0.5
R5    Accuracy: 1 / 2 = 0.5
R6    Accuracy: 1 / 1 = 1.0


(1280, 0.6349206349206349)

In [54]:
basic_model = mmm.BasicPredictor()
basic_tourney = mmc.Tournament(seeds_2001, slots_2001, basic_model, include_scoring_dif=False)

basic_tourney.score_tournament(actual_tourney, print_res=True)

Total Points  : 1230

Total Accuracy: 38 / 63 = 0.603174603175
R1    Accuracy: 19 / 32 = 0.59375
R2    Accuracy: 10 / 16 = 0.625
R3    Accuracy: 5 / 8 = 0.625
R4    Accuracy: 2 / 4 = 0.5
R5    Accuracy: 1 / 2 = 0.5
R6    Accuracy: 1 / 1 = 1.0


(1230, 0.6031746031746031)